In [125]:
!pip install pyspark

In [126]:
from pyspark.sql import SparkSession
import pyspark
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","10g")\
        .getOrCreate()

In [127]:
traffic_crashes=spark.read.option('header',True).option("inferSchema", True).csv('/content/drive/MyDrive/Chicago dataset/Traffic_Crashes_-_Crashes.csv')

In [128]:
traffic_crashes.show(5)


+--------------------+--------+----------------+--------------------+------------------+----------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------+------------------+--------------------+-----------+-----------+--------------------+----------------------+------------------+-------------+-------------+--------------------+-----------------------+----------------------+---------+----------------+----------------+------------------+--------------+------------------+---------+-----------+--------------+-----------------+---------+--------------------+--------------+--------------+-----------------------+---------------------------+-----------------------------+----------------------+----------------+----------+-----------------+-----------+------------+-------------+--------------------+
|     CRASH_RECORD_ID|   RD_NO|CRASH_DATE_EST_I|          CRASH_DATE|POSTED_SPEED_LIMIT|TRAFFIC_CONTROL_DEVICE|    DEVICE_CONDITIO

In [130]:
traffic_crashes.summary().show()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [131]:
traffic_crashes.printSchema()

root
 |-- CRASH_RECORD_ID: string (nullable = true)
 |-- RD_NO: string (nullable = true)
 |-- CRASH_DATE_EST_I: string (nullable = true)
 |-- CRASH_DATE: string (nullable = true)
 |-- POSTED_SPEED_LIMIT: integer (nullable = true)
 |-- TRAFFIC_CONTROL_DEVICE: string (nullable = true)
 |-- DEVICE_CONDITION: string (nullable = true)
 |-- WEATHER_CONDITION: string (nullable = true)
 |-- LIGHTING_CONDITION: string (nullable = true)
 |-- FIRST_CRASH_TYPE: string (nullable = true)
 |-- TRAFFICWAY_TYPE: string (nullable = true)
 |-- LANE_CNT: integer (nullable = true)
 |-- ALIGNMENT: string (nullable = true)
 |-- ROADWAY_SURFACE_COND: string (nullable = true)
 |-- ROAD_DEFECT: string (nullable = true)
 |-- REPORT_TYPE: string (nullable = true)
 |-- CRASH_TYPE: string (nullable = true)
 |-- INTERSECTION_RELATED_I: string (nullable = true)
 |-- NOT_RIGHT_OF_WAY_I: string (nullable = true)
 |-- HIT_AND_RUN_I: string (nullable = true)
 |-- DAMAGE: string (nullable = true)
 |-- DATE_POLICE_NOTIFIED

In [132]:
traffic_crashes.count()

610025

In [133]:
from pyspark.sql.functions import date_format
from pyspark.sql.functions import unix_timestamp


traffic_crashes=traffic_crashes.withColumn('Date',(date_format(unix_timestamp("CRASH_DATE", "MM/dd/yyyy hh:mm:ss a").cast("timestamp"), "MM/dd/yyyy")))
traffic_crashes.withColumn('year',date_format('Date','yyyy'))

DataFrame[CRASH_RECORD_ID: string, RD_NO: string, CRASH_DATE_EST_I: string, CRASH_DATE: string, POSTED_SPEED_LIMIT: int, TRAFFIC_CONTROL_DEVICE: string, DEVICE_CONDITION: string, WEATHER_CONDITION: string, LIGHTING_CONDITION: string, FIRST_CRASH_TYPE: string, TRAFFICWAY_TYPE: string, LANE_CNT: int, ALIGNMENT: string, ROADWAY_SURFACE_COND: string, ROAD_DEFECT: string, REPORT_TYPE: string, CRASH_TYPE: string, INTERSECTION_RELATED_I: string, NOT_RIGHT_OF_WAY_I: string, HIT_AND_RUN_I: string, DAMAGE: string, DATE_POLICE_NOTIFIED: string, PRIM_CONTRIBUTORY_CAUSE: string, SEC_CONTRIBUTORY_CAUSE: string, STREET_NO: int, STREET_DIRECTION: string, STREET_NAME: string, BEAT_OF_OCCURRENCE: int, PHOTOS_TAKEN_I: string, STATEMENTS_TAKEN_I: string, DOORING_I: string, WORK_ZONE_I: string, WORK_ZONE_TYPE: string, WORKERS_PRESENT_I: string, NUM_UNITS: int, MOST_SEVERE_INJURY: string, INJURIES_TOTAL: int, INJURIES_FATAL: int, INJURIES_INCAPACITATING: int, INJURIES_NON_INCAPACITATING: int, INJURIES_REPOR

In [134]:
traffic_crashes.show(20)

+--------------------+--------+----------------+--------------------+------------------+----------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------+------------------+--------------------+-----------+--------------------+--------------------+----------------------+------------------+-------------+-------------+--------------------+-----------------------+----------------------+---------+----------------+----------------+------------------+--------------+------------------+---------+-----------+--------------+-----------------+---------+--------------------+--------------+--------------+-----------------------+---------------------------+-----------------------------+----------------------+----------------+----------+-----------------+-----------+------------+-------------+--------------------+----------+
|     CRASH_RECORD_ID|   RD_NO|CRASH_DATE_EST_I|          CRASH_DATE|POSTED_SPEED_LIMIT|TRAFFIC_CONTROL_DEVICE

In [135]:
traffic_crashes=traffic_crashes.withColumn('year',(date_format(unix_timestamp("CRASH_DATE", "MM/dd/yyyy hh:mm:ss a").cast("timestamp"), "yyyy")))
traffic_crashes=traffic_crashes.withColumn('month',(date_format(unix_timestamp("CRASH_DATE", "MM/dd/yyyy hh:mm:ss a").cast("timestamp"), "MM")))

In [136]:
traffic_crashes.show(5)

+--------------------+--------+----------------+--------------------+------------------+----------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------+------------------+--------------------+-----------+-----------+--------------------+----------------------+------------------+-------------+-------------+--------------------+-----------------------+----------------------+---------+----------------+----------------+------------------+--------------+------------------+---------+-----------+--------------+-----------------+---------+--------------------+--------------+--------------+-----------------------+---------------------------+-----------------------------+----------------------+----------------+----------+-----------------+-----------+------------+-------------+--------------------+----------+----+-----+
|     CRASH_RECORD_ID|   RD_NO|CRASH_DATE_EST_I|          CRASH_DATE|POSTED_SPEED_LIMIT|TRAFFIC_CONTROL_DEVI

In [137]:
from pyspark.sql.functions import count
yearly_count=traffic_crashes.groupBy('year').agg(count('CRASH_RECORD_ID'))
monthly_count=traffic_crashes.groupBy('year','month').agg(count('CRASH_RECORD_ID'))

In [138]:
yearly_count.show(20)
monthly_count.show(20)

+----+----------------------+
|year|count(CRASH_RECORD_ID)|
+----+----------------------+
|2016|                 44297|
|2020|                 92106|
|2019|                117761|
|2017|                 83786|
|2014|                     6|
|2018|                118950|
|2022|                 34506|
|2021|                108784|
|2015|                  9828|
|2013|                     1|
+----+----------------------+

+----+-----+----------------------+
|year|month|count(CRASH_RECORD_ID)|
+----+-----+----------------------+
|2016|   05|                  3091|
|2015|   09|                  1524|
|2017|   08|                  7685|
|2020|   11|                  6963|
|2020|   10|                  8357|
|2018|   01|                  9532|
|2018|   07|                 10367|
|2019|   04|                  9446|
|2015|   11|                  2606|
|2018|   04|                  9648|
|2021|   10|                 10272|
|2018|   10|                 10402|
|2017|   01|                  4363|
|20

In [139]:
monthly_count.select('*').where(monthly_count['year']=='2022').show(12)

+----+-----+----------------------+
|year|month|count(CRASH_RECORD_ID)|
+----+-----+----------------------+
|2022|   01|                  8202|
|2022|   02|                  7962|
|2022|   04|                  8444|
|2022|   05|                  1351|
|2022|   03|                  8547|
+----+-----+----------------------+



In [140]:
temp=monthly_count.select('*').where(monthly_count['year']=='2016')

In [141]:
import pyspark.sql.functions as F   
temp.groupBy('year').agg(F.sum('count(CRASH_RECORD_ID)')).show(10)

+----+---------------------------+
|year|sum(count(CRASH_RECORD_ID))|
+----+---------------------------+
|2016|                      44297|
+----+---------------------------+



In [142]:
traffic_crashes.show(4)

+--------------------+--------+----------------+--------------------+------------------+----------------------+--------------------+-----------------+------------------+--------------------+--------------------+--------+------------------+--------------------+-----------+-----------+--------------------+----------------------+------------------+-------------+-------------+--------------------+-----------------------+----------------------+---------+----------------+----------------+------------------+--------------+------------------+---------+-----------+--------------+-----------------+---------+--------------------+--------------+--------------+-----------------------+---------------------------+-----------------------------+----------------------+----------------+----------+-----------------+-----------+------------+-------------+--------------------+----------+----+-----+
|     CRASH_RECORD_ID|   RD_NO|CRASH_DATE_EST_I|          CRASH_DATE|POSTED_SPEED_LIMIT|TRAFFIC_CONTROL_DEVICE

In [143]:
traffic_crashes_filtered=traffic_crashes.na.drop(subset=["CRASH_DATE","LATITUDE","LONGITUDE"])


In [144]:
traffic_crashes_filtered=traffic_crashes_filtered.filter(traffic_crashes_filtered.LATITUDE!=0.0)
traffic_crashes_filtered=traffic_crashes_filtered.filter(traffic_crashes_filtered.LONGITUDE!=0.0)

In [145]:
import pyspark

In [146]:
import math
def getBins(val):
  val=float(val)
  #print(val)
  bin=math.floor(val/0.15)*0.15
  return bin

getBinsUDF=pyspark.sql.functions.udf(getBins)

In [163]:
#getBins("1")
temp=traffic_crashes_filtered.withColumn('latBin',getBinsUDF('LATITUDE'))
temp=temp.withColumn('longBin',getBinsUDF('LONGITUDE'))

In [148]:
temp.select('*').where((temp['latBin']==0.0) & (temp['longBin']==0.0)).show()

+---------------+-----+----------------+----------+------------------+----------------------+----------------+-----------------+------------------+----------------+---------------+--------+---------+--------------------+-----------+-----------+----------+----------------------+------------------+-------------+------+--------------------+-----------------------+----------------------+---------+----------------+-----------+------------------+--------------+------------------+---------+-----------+--------------+-----------------+---------+------------------+--------------+--------------+-----------------------+---------------------------+-----------------------------+----------------------+----------------+----------+-----------------+-----------+--------+---------+--------+----+----+-----+------+-------+
|CRASH_RECORD_ID|RD_NO|CRASH_DATE_EST_I|CRASH_DATE|POSTED_SPEED_LIMIT|TRAFFIC_CONTROL_DEVICE|DEVICE_CONDITION|WEATHER_CONDITION|LIGHTING_CONDITION|FIRST_CRASH_TYPE|TRAFFICWAY_TYPE|LANE_

In [169]:
temp.select('DAMAGE').distinct().show()

+------+
|DAMAGE|
+------+
|  1500|
|  1000|
|   500|
+------+



In [168]:
temp=temp.withColumn('DAMAGE',F.when(temp.DAMAGE=="OVER $1,500",1500).otherwise(temp.DAMAGE))
temp=temp.withColumn('DAMAGE',F.when(temp.DAMAGE=="$501 - $1,500",1000).otherwise(temp.DAMAGE))
temp=temp.withColumn('DAMAGE',F.when(temp.DAMAGE=="$500 OR LESS",500).otherwise(temp.DAMAGE))


In [149]:
temp.select('HIT_AND_RUN_I').distinct().show()

+-------------+
|HIT_AND_RUN_I|
+-------------+
|         null|
|            Y|
|            N|
+-------------+



In [175]:
from pyspark.sql.functions import countDistinct
#agg(count(when(col("col_2") == 'X',1)))
temp.groupBy('year','month','latBin','longBin').agg(count(F.col('CRASH_RECORD_ID')),F.sum(F.col('INJURIES_TOTAL')),count(F.when(F.col('HIT_AND_RUN_I')=='Y',1)),F.sum(F.col('DAMAGE'),)).show()

+----+-----+------------------+------------------+----------------------+-------------------+-----------------------------------------------+-----------+
|year|month|            latBin|           longBin|count(CRASH_RECORD_ID)|sum(INJURIES_TOTAL)|count(CASE WHEN (HIT_AND_RUN_I = Y) THEN 1 END)|sum(DAMAGE)|
+----+-----+------------------+------------------+----------------------+-------------------+-----------------------------------------------+-----------+
|2022|   03|41.699999999999996|-87.89999999999999|                   125|                 13|                                             36|   158000.0|
|2019|   12|              42.0|-87.89999999999999|                    22|                  4|                                              5|    30500.0|
|2021|   03|              42.0|            -87.75|                   120|                 20|                                             40|   144500.0|
|2017|   03|             41.55|            -87.75|                   119|   

+------------------+------------------+
|            latBin|           longBin|
+------------------+------------------+
|41.699999999999996|            -87.75|
|             41.85|-87.89999999999999|
|             41.55|             -87.6|
|              42.0|            -88.05|
|41.699999999999996|             -87.6|
|             41.55|            -87.75|
|              42.0|            -87.75|
|             41.85|            -87.75|
|41.699999999999996|-87.89999999999999|
|             41.85|            -88.05|
|              42.0|-87.89999999999999|
+------------------+------------------+

